In [1]:
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

load_dotenv()

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
ai_message = model.invoke([HumanMessage(content="Hi! I'm Dan.")])

print(ai_message.content)

Hi Dan! It's great to meet you. How can I help you today?


In [2]:
ai_message = model.invoke([
    HumanMessage(content="Hi! I'm Dan."),
    AIMessage(content="Hi Dan! It's great to meet you. How can I help you today?"),
    HumanMessage(content="What's my name?"),
])

print(ai_message.content)

Your name is Dan!


In [10]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You talk like a pirate."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [3]:
config = { "configurable": {"thread_id": "abc123"}}

In [12]:
output = app.invoke({"messages": [HumanMessage(content="Hi! I'm Dan.")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Dan! A fine name for a landlubber, or perhaps a future scallywag o' the seas! What brings ye to these here waters?


In [13]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Blast me barnacles! Ye just told ol' Cap'n Craggybeard yer name, ye did! Ye be **Dan**, if me memory serves me right and true! Did ye take a knock to the head, or be ye just testin' me wits, eh?


In [8]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc456"}}

output = app.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Dan.")], "language": "German"}, config
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hallo, Dan!


In [22]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Dein Name ist Dan.


In [5]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=55,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [6]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])

    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [39]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. As an AI, I don't have access to personal information about you unless you choose to tell me.


In [41]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked "What's 2 + 2?"


In [ ]:
from langchain_core.messages import AIMessageChunk

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]

# output = app.invoke(
#     {"messages": input_messages, "language": language},
#     config,
# )
# output["messages"][-1].pretty_print()

for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language}, config, stream_mode="messages"
):
    # if isinstance(chunk, AIMessageChunk):
    #   print(chunk.content, end="|")
    print(chunk.content)

Hi Todd! Here's one for you:

Why don't scientists trust atoms?

Because they make up everything!
1
